In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    )
template = """
    You are a helpful AI talking to human.

    {chat_history}
    Human: {question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question = "My name is Nico.")




> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to human.

    
    Human: My name is Nico.
    You:


> Finished chain.


'Nice to meet you, Nico! How can I assist you today?'

In [6]:
chain.predict(question = "I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to human.

    System: The human introduces themselves as Nico. The AI responds by saying it's nice to meet Nico and asks how it can assist them today.
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is Nico.
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is Nico.
    Human: I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with information or tasks related to Seoul?"

In [5]:
chain.predict(question = "What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to human.

    Human: My name is Nico.
AI: Nice to meet you, Nico! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
Human: What is my name?
AI: Your name is Nico.
Human: I live in Seoul
AI: That's great to know! How can I assist you with information or tasks related to Seoul?
    Human: What is my name?
    You:


> Finished chain.


'Your name is Nico.'